## implementation stollen from https://blog.paperspace.com/beginners-guide-to-boltzmann-machines-pytorch/

In [1]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torchvision'

In [2]:
batch_size = 64
# train_loader = torch.utils.data.DataLoader(
# datasets.MNIST('./data',
#     train=True,
#     download = True,
#     transform = transforms.Compose(
#         [transforms.ToTensor()])
#      ),
#      batch_size=batch_size
# )

# test_loader = torch.utils.data.DataLoader(
# datasets.MNIST('./data',
#     train=False,
#     transform=transforms.Compose(
#     [transforms.ToTensor()])
#     ),
#     batch_size=batch_size)

def gen_xor_training(N):
    data   = np.random.randint(0, 1, (N, 2))
    target = np.logical_xor(data[:, 0:1], data[:, 1:2])
    return data,target

In [5]:
class RBM(nn.Module):
   def __init__(self,
               n_vis,
               n_hin,
               k=5):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hin,n_vis)*1e-2)
        self.v_bias = nn.Parameter(torch.zeros(n_vis))
        self.h_bias = nn.Parameter(torch.zeros(n_hin))
        self.k = k
    
   def sample_from_p(self,p):
       return F.relu(torch.sign(p - Variable(torch.rand(p.size()))))
    
   def v_to_h(self,v):
        p_h = F.sigmoid(F.linear(v,self.W,self.h_bias))
        sample_h = self.sample_from_p(p_h)
        return p_h,sample_h
    
   def h_to_v(self,h):
        p_v = F.sigmoid(F.linear(h,self.W.t(),self.v_bias))
        sample_v = self.sample_from_p(p_v)
        return p_v,sample_v
        
   def forward(self,v):
        pre_h1,h1 = self.v_to_h(v)
        
        h_ = h1
        for _ in range(self.k):
            pre_v_,v_ = self.h_to_v(h_)
            pre_h_,h_ = self.v_to_h(v_)
        
        return v,v_
    
   def free_energy(self,v):
        vbias_term = v.mv(self.v_bias)
        wx_b = F.linear(v,self.W,self.h_bias)
        hidden_term = wx_b.exp().add(1).log().sum(1)
        return (-hidden_term - vbias_term).mean()



In [11]:
rbm = RBM(3, 2, k=1)
train_op = optim.SGD(rbm.parameters(),0.01)
losses=[]

for epoch in range(10):
    loss_ = []
    #for _, (data,target) in enumerate(train_loader):
    for data,target in zip(*gen_xor_training(batch_size)):
        #data = Variable(data.view(-1,784))
        data = Variable(data)
        sample_data = data.bernoulli()
        
        v,v1 = rbm(sample_data)
        loss = rbm.free_energy(v) - rbm.free_energy(v1)
        loss_.append(loss.data)
        train_op.zero_grad()
        loss.backward()
        train_op.step()
    losses.append(np.mean(loss_))
    print("Training loss for {} epoch: {}".format(epoch, np.mean(loss_)))


TypeError: Variable data has to be a tensor, but got numpy.ndarray

In [12]:
plt.plot(losses)

NameError: name 'plt' is not defined

In [9]:
def show_and_save(file_name,img):
    npimg = np.transpose(img.numpy(),(1,2,0))
    f = "./%s.png" % file_name
    plt.imshow(npimg)
    plt.imsave(f,npimg)


In [10]:
show_and_save("real",make_grid(v.view(100,1,28,28).data))


RuntimeError: shape '[100, 1, 28, 28]' is invalid for input of size 25088

In [ ]:
show_and_save("generate",make_grid(v1.view(32,1,28,28).data))


NameError: name 'show_and_save' is not defined